In [2]:
import pandas as pd

takes = pd.read_csv("Takes_Feb25.csv")
watch_data = pd.read_csv("Watch-Data_Feb25.csv")

takes_copy = takes.copy()
watch_data_copy = watch_data.copy()

In [3]:
takes_copy.head()

,Activity,Assignment,Cohort,Prompt,Public View,Save Take,Submit Video,Take Label,Take Length,Task,Token,User,Video URL,Creation Date,Modified Date,Slug,Creator,unique id
0,Your Story,NaN,February 2022 Test,NaN,NaN,NaN,yes,NaN,NaN,NaN,2542111fe6918c176f2c231b278461a3,lauren@laurenlizardo.com,NaN,"Feb 17, 2022 11:09 pm","Feb 17, 2022 11:12 pm",NaN,(deleted thing),1645157397692x647200668964618200
1,Your Story,NaN,February 2022 Test,NaN,NaN,NaN,yes,NaN,NaN,NaN,a55e48ca94cf58b32822c14537bd0a94,lauren@laurenlizardo.com,NaN,"Feb 17, 2022 11:12 pm","Feb 17, 2022 11:12 pm",NaN,(deleted thing),1645157541961x652916174365655000
2,Bonus Scene: Teamwork,NaN,February 2022 Test,NaN,NaN,yes,yes,Test 1,NaN,NaN,NaN,luckeyalexis@yahoo.com,//s3.amazonaws.com/appforest_uf/f1646695709353...,"Mar 7, 2022 6:28 pm","Mar 7, 2022 6:28 pm",NaN,(deleted thing),1646695710597x552768075757256700
3,Freestyle!,NaN,February 2022 Test,NaN,NaN,yes,yes,Test 2,NaN,NaN,NaN,luckeyalexis@yahoo.com,//s3.amazonaws.com/appforest_uf/f1646695940928...,"Mar 7, 2022 6:32 pm","Mar 7, 2022 6:33 pm",NaN,(deleted thing),1646695942253x700120630636838900
4,Video Recorder Test,NaN,March 2022 Test,NaN,NaN,yes,yes,TEst 1,NaN,NaN,NaN,ryan+p@betteron.video,//s3.amazonaws.com/appforest_uf/f1646841481243...,"Mar 9, 2022 10:58 am","Mar 9, 2022 10:58 am",NaN,ryan+p@betteron.video,1646841482111x738832509098524700


In [4]:
watch_data_copy.head()

,Activity,API-Timeout-ID,API-Popup-ID,Participant,Popup,Status,Stop Time
0,NaN,NaN,NaN,jboyer@redhat.com,NaN,Start,"Oct 18, 2023 11:14 am"
1,NaN,NaN,NaN,jboyer@redhat.com,NaN,Start,"Oct 18, 2023 11:18 am"
2,NaN,NaN,NaN,jboyer@redhat.com,NaN,Start,"Oct 18, 2023 11:21 am"
3,NaN,NaN,NaN,dramseur@redhat.com,NaN,Start,"Oct 18, 2023 12:27 pm"
4,NaN,NaN,NaN,dramseur@redhat.com,NaN,Start,"Oct 18, 2023 12:28 pm"


In [5]:
takes.columns

Index(['Activity', 'Assignment', 'Cohort', 'Prompt', 'Public View',
       'Save Take', 'Submit Video', 'Take Label', 'Take Length', 'Task',
       'Token', 'User', 'Video URL', 'Creation Date', 'Modified Date', 'Slug',
       'Creator', 'unique id'],
      dtype='object')

In [6]:
watch_data.columns

Index(['Activity', 'API-Timeout-ID', 'API-Popup-ID', 'Participant', 'Popup',
       'Status', 'Stop Time'],
      dtype='object')

In [7]:
watch_data.fillna({'Activity': 'Unknown', 'Popup': 'None', 'Status': 'Unknown'}, inplace=True)
takes.fillna({'Activity': 'Unknown', 'Assignment': 'Unknown', 'Cohort': 'Unknown', 'Save Take': 'no', 'Submit Video': 'no'}, inplace=True)


takes['Save Take'] = takes['Save Take'].apply(lambda x: 1 if x == 'yes' else 0)
takes['Submit Video'] = takes['Submit Video'].apply(lambda x: 1 if x == 'yes' else 0)

takes['Take Length'] = pd.to_numeric(takes['Take Length'], errors='coerce')
takes['Take Length'] = takes['Take Length'].fillna(0) 

watch_data[['API-Timeout-ID', 'API-Popup-ID']] = watch_data[['API-Timeout-ID', 'API-Popup-ID']].fillna(0)

In [8]:
date_form = "%b %d, %Y %I:%M %p"

watch_data['Stop Time'] = pd.to_datetime(watch_data['Stop Time'], errors='coerce', format=date_form)
takes['Creation Date'] = pd.to_datetime(takes['Creation Date'], errors='coerce', format=date_form)
takes['Modified Date'] = pd.to_datetime(takes['Modified Date'], errors='coerce', format=date_form)

In [9]:
watch_user_summary = watch_data.groupby('Participant').agg(
    activity_count=('Activity', 'count'),
    popup_interactions=('Popup', lambda x: (x.notna() & (x != 'None')).sum())
).reset_index()

In [10]:
takes_user_summary = takes.groupby('User').agg(
    total_submissions=('Submit Video', 'sum'),
    total_saves=('Save Take', 'sum'),
    avg_take_length=('Take Length', 'mean')  # Average take length per user
).reset_index()

In [11]:
user_summary = pd.merge(takes_user_summary, watch_user_summary,
                        left_on='User', right_on='Participant', how='outer').drop(columns=['Participant'])

In [12]:
user_summary.head(15)

,User,total_submissions,total_saves,avg_take_length,activity_count,popup_interactions
0,NaN,NaN,NaN,NaN,1.0,1.0
1,1kgilliland@gmail.com,28.0,28.0,0.000000,NaN,NaN
2,850jcwalker@gmail.com,1.0,1.0,0.000000,NaN,NaN
3,a.llewellyn@redhat.com,2.0,1.0,8.500000,2.0,2.0
4,aaron@seatzapp.com,10.0,10.0,0.000000,NaN,NaN
5,ab1043@duke.edu,0.0,0.0,1.000000,1.0,1.0
6,ab883@duke.edu,3.0,4.0,3.166667,6.0,6.0
7,abhinav@automattic.com,17.0,18.0,43.961538,19.0,19.0
8,abohra@us.ibm.com,23.0,24.0,72.560000,24.0,24.0
9,abraham.waita@automattic.com,23.0,23.0,0.000000,NaN,NaN


In [13]:
weights = {
    'total_submissions': 5,  
    'total_saves': 3,        
    'avg_take_length': 2,
    'activity_count': 4,
    'popup_interactions': 1
}

In [14]:
user_summary['ranking_score'] = (
    user_summary['total_submissions'] * weights['total_submissions'] +
    user_summary['total_saves'] * weights['total_saves'] +
    user_summary['avg_take_length'] * weights['avg_take_length'] +
    user_summary['activity_count'] * weights['activity_count'] +
    user_summary['popup_interactions'] * weights['popup_interactions']
)

In [15]:
user_summary = user_summary.sort_values(by='ranking_score', ascending=False)

In [16]:
user_summary.head()

,User,total_submissions,total_saves,avg_take_length,activity_count,popup_interactions,ranking_score
175,dramseur@redhat.com,34.0,71.0,103.329787,41.0,0.0,753.659574
559,shevene.lovindier@automattic.com,31.0,43.0,116.282609,44.0,44.0,736.565217
551,seiji.hara@automattic.com,29.0,36.0,67.217949,63.0,63.0,702.435897
581,sudeep.baral@automattic.com,23.0,24.0,143.864865,44.0,44.0,694.729730
482,ramona.nemeth-csoka@automattic.com,25.0,31.0,111.538462,49.0,49.0,686.076923
